# 1. Introduction

In this project, we'll work with a data set of submissions to popular technology site Hacker News.

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts/download), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

- `id`: The unique identifier from Hacker News for the post
- `title`: The title of the post
- `url`: The URL that the posts links to, if it the post has a URL
- `num_points`: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- `num_comments`: The number of comments that were made on the post
- `author`: The username of the person who submitted the post
- `created_at`: The date and time at which the post was submitted

# 2. Removing Headers from a List of Lists

In [1]:
from csv import reader
import datetime as dt

file = open("hacker_news.csv")
readit = reader(file)
listit = list(readit)
hn = listit[1:]
header = listit[0]

print(hn[:3])
print(header)

[['12579008', 'You have two days to comment if you want stem cells to be classified as your own', 'http://www.regulations.gov/document?D=FDA-2015-D-3719-0018', '1', '0', 'altstar', '9/26/2016 3:26'], ['12579005', 'SQLAR  the SQLite Archiver', 'https://www.sqlite.org/sqlar/doc/trunk/README.md', '1', '0', 'blacksqr', '9/26/2016 3:24'], ['12578997', 'What if we just printed a flatscreen television on the side of our boxes?', 'https://medium.com/vanmoof/our-secrets-out-f21c1f03fdc8#.ietxmez43', '1', '0', 'pavel_lishin', '9/26/2016 3:19']]
['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


# 3. Extracting Ask HN and Show HN Posts

In [2]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1].lower()
    if title.startswith("ask hn"):
        ask_posts.append(row)
    elif title.startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))


9139
10158
273822


# 4. Calculating the Average Number of Comments for Ask HN and Show HN Posts

In [3]:
def total_comments(dataset, comm_ind):
    sum_comments = 0
    for row in dataset:
        comm = int(row[comm_ind])
        sum_comments += comm
    return sum_comments


total_ask_comments = total_comments(ask_posts, 4)
total_show_comments = total_comments(show_posts, 4)

print("Sum of Ask Comments: ")
print(total_ask_comments)
print("Average Comments for each Ask Post: ")
avg_ask_comments = round(total_ask_comments/len(ask_posts))
print(avg_ask_comments)

print("Sum of Show Comments: ")
print(total_show_comments)
print("Average Comments for each Show Post: ")
avg_show_comments = round(total_show_comments/len(show_posts))
print(avg_show_comments)
    

Sum of Ask Comments: 
94986
Average Comments for each Ask Post: 
10
Sum of Show Comments: 
49633
Average Comments for each Show Post: 
5


# 5. Finding the Amount of Ask Posts and Comments by Hour Created

In [4]:
def posts_per_hour(dataset, time, comms):
    result_list = []
    for row in dataset:
        create = row[time]
        comm = int(row[comms])
        result_list.append([create,comm])
    return result_list

pph = posts_per_hour(ask_posts,6,4 )
print(pph[:3])

[['9/26/2016 2:53', 7], ['9/26/2016 1:17', 3], ['9/25/2016 22:57', 0]]


In [5]:
counts_by_hour = {}
comments_by_hour = {}

for row in pph:
    date = row[0]
    comm = row[1]
    datum = dt.datetime.strptime(date, "%m/%d/%Y %H:%M")
    hour = datum.strftime("%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = comm
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += comm
    
print(counts_by_hour)
print(comments_by_hour)

{'02': 269, '01': 282, '22': 383, '21': 518, '19': 552, '17': 587, '15': 646, '14': 513, '13': 444, '11': 312, '10': 282, '09': 222, '07': 226, '03': 271, '23': 343, '20': 510, '16': 579, '08': 257, '00': 301, '18': 614, '12': 342, '04': 243, '06': 234, '05': 209}
{'02': 2996, '01': 2089, '22': 3372, '21': 4500, '19': 3954, '17': 5547, '15': 18525, '14': 4972, '13': 7245, '11': 2797, '10': 3013, '09': 1477, '07': 1585, '03': 2154, '23': 2297, '20': 4462, '16': 4466, '08': 2362, '00': 2277, '18': 4877, '12': 4234, '04': 2360, '06': 1587, '05': 1838}


# 6. Calculating the Average Number of Comments for Ask HN Posts by Hour

In [6]:
avg_by_hour = []

for key in counts_by_hour:
    avg_by_hour.append([key, round(comments_by_hour[key]/counts_by_hour[key])])

print(avg_by_hour)
print(len(comments_by_hour))

[['02', 11], ['01', 7], ['22', 9], ['21', 9], ['19', 7], ['17', 9], ['15', 29], ['14', 10], ['13', 16], ['11', 9], ['10', 11], ['09', 7], ['07', 7], ['03', 8], ['23', 7], ['20', 9], ['16', 8], ['08', 9], ['00', 8], ['18', 8], ['12', 12], ['04', 10], ['06', 7], ['05', 9]]
24


# 7. Sorting and Printing Values from a List of Lists

In [7]:
swap_avg_by_hour = []

for row in avg_by_hour:
    first = row[1]
    second = row[0]
    swap_avg_by_hour.append([first,second])

print(swap_avg_by_hour)

[[11, '02'], [7, '01'], [9, '22'], [9, '21'], [7, '19'], [9, '17'], [29, '15'], [10, '14'], [16, '13'], [9, '11'], [11, '10'], [7, '09'], [7, '07'], [8, '03'], [7, '23'], [9, '20'], [8, '16'], [9, '08'], [8, '00'], [8, '18'], [12, '12'], [10, '04'], [7, '06'], [9, '05']]


In [8]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
top5 = sorted_swap[:6]
print("Top 5 Hours for Ask Posts Comments")

def print_avg(dataset, av, hr, txt):
    for row in dataset:
        avg = row[av]
        hour = row[hr]
        formatted = dt.datetime.strptime(hour, "%H")
        only_hour = dt.datetime.strftime(formatted, "%H:%M")
        template = str.format("{time}: {comms} average "+ txt + " per post", time = only_hour, comms = avg)
        print(template)

print_avg(top5, 0,1, "comments")

Top 5 Hours for Ask Posts Comments
15:00: 29 average comments per post
13:00: 16 average comments per post
12:00: 12 average comments per post
10:00: 11 average comments per post
02:00: 11 average comments per post
14:00: 10 average comments per post


# 8. Next Steps

- Determine if show or ask posts receive more points on average.
- Determine if posts created at a certain time are more likely to receive more points.
- Compare your results to the average number of comments and points other posts receive.
- Use Dataquest's data science project style guide to format your project.

In [9]:
total_points_ask = total_comments(ask_posts, 3)
total_points_show = total_comments(show_posts, 3)
print("Total Points for Ask Posts:")
print(total_points_ask)
print("Average Points for Ask Posts:")
avg_points_ask = total_points_ask/len(ask_posts)
print(round(avg_points_ask))
print("Total Points for Show Posts:")
print(total_points_show)
print("Average Points for Show Posts:")
avg_points_show = total_points_show/len(show_posts)
print(round(avg_points_show))

Total Points for Ask Posts:
103378
Average Points for Ask Posts:
11
Total Points for Show Posts:
150781
Average Points for Show Posts:
15


In [10]:
show_per_hour = posts_per_hour(show_posts, 6,3)
print(show_per_hour[:6])

[['9/26/2016 0:36', 2], ['9/26/2016 0:01', 1], ['9/25/2016 23:44', 1], ['9/25/2016 23:17', 2], ['9/25/2016 20:06', 1], ['9/25/2016 19:06', 1]]


In [11]:
points_by_hour = {}

for row in show_per_hour:
    date = row[0]
    point = row[1]
    datum = dt.datetime.strptime(date, "%m/%d/%Y %H:%M")
    hour = datum.strftime("%H")
    if hour not in points_by_hour:
        points_by_hour[hour] = point
    else:
        points_by_hour[hour] += point
    
print(counts_by_hour)
print(points_by_hour)

{'02': 269, '01': 282, '22': 383, '21': 518, '19': 552, '17': 587, '15': 646, '14': 513, '13': 444, '11': 312, '10': 282, '09': 222, '07': 226, '03': 271, '23': 343, '20': 510, '16': 579, '08': 257, '00': 301, '18': 614, '12': 342, '04': 243, '06': 234, '05': 209}
{'00': 4291, '23': 5060, '20': 6948, '19': 8928, '18': 9935, '16': 11487, '14': 10503, '10': 4303, '09': 3762, '08': 4640, '06': 3071, '03': 2168, '21': 5990, '17': 10563, '15': 11657, '11': 7742, '07': 3303, '04': 2707, '13': 10381, '12': 10787, '01': 2931, '22': 5026, '02': 2764, '05': 1834}


In [12]:
avg_points_by_hour = []

for key in points_by_hour:
    avg_points_by_hour.append([key, round(points_by_hour[key]/len(points_by_hour))])

print(avg_points_by_hour)

[['00', 179], ['23', 211], ['20', 290], ['19', 372], ['18', 414], ['16', 479], ['14', 438], ['10', 179], ['09', 157], ['08', 193], ['06', 128], ['03', 90], ['21', 250], ['17', 440], ['15', 486], ['11', 323], ['07', 138], ['04', 113], ['13', 433], ['12', 449], ['01', 122], ['22', 209], ['02', 115], ['05', 76]]


In [13]:
swap_avg_points_by_hour = []

for row in avg_points_by_hour:
    first = row[1]
    second = row[0]
    swap_avg_points_by_hour.append([first,second])

print(swap_avg_points_by_hour)

[[179, '00'], [211, '23'], [290, '20'], [372, '19'], [414, '18'], [479, '16'], [438, '14'], [179, '10'], [157, '09'], [193, '08'], [128, '06'], [90, '03'], [250, '21'], [440, '17'], [486, '15'], [323, '11'], [138, '07'], [113, '04'], [433, '13'], [449, '12'], [122, '01'], [209, '22'], [115, '02'], [76, '05']]


In [14]:
sorted_points = sorted(swap_avg_points_by_hour, reverse=True)
top5_points = sorted_points[:6]
print("Top 5 Hours for Show Posts Highest Scores:")
print_avg(top5_points, 0,1, "points")


Top 5 Hours for Show Posts Highest Scores:
15:00: 486 average points per post
16:00: 479 average points per post
12:00: 449 average points per post
17:00: 440 average points per post
14:00: 438 average points per post
13:00: 433 average points per post
